# Рекомендация тарифов

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — она выполнена в предыдущем проекте.

Построим модель с максимально большим значением *accuracy*. Проверим *accuracy* на тестовой выборке.

## 1. Откроем и изучим файл

In [1]:
import pandas as pd
from scipy import stats as st
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'C:\Users\Айболит\Desktop\DataFrames\projects\best_tariff_recomenfation\users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.duplicated().sum()

0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [7]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

In [8]:
len(df)

3214

__Вывод__

Датафрейм представлен нам без пропусков в данных и дубликатов. Представлены три основных признака и тариф, который наша модель и будет предсказывать исходя из признаков. Причем имеется скошенность данных в сторону тарифа Смарт - примерно 70% абонентов и 30% абонентов тарифа Ультра.

Мы уже знаем, по предыдущему проекту, что есть абоненты, которые пользуются тарифом Смарт, но часто превышают лимит и переплачивают больше, чем если бы они пользовались тарифом Ультра. Но тут выбросов быть не может, т.к. мы не знаем предпочтений, убеждений абонента, род деятельности. И модель должна учитывать разный спектр пользователей.

## 2. Разобьем данные на выборки

Предлогаю работать в режиме с тремя выборками - тренировочная, валидационная и тестовая.

In [9]:
df_train2, df_general2 = train_test_split(df, test_size=0.40, random_state=222)
df_valid2, df_test2 = train_test_split(df_general2, test_size=0.50, random_state=222)

In [10]:
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 346 to 3206
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [11]:
df_valid2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2359 to 2259
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [12]:
df_test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2750 to 1573
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [13]:
features_train_2 = df_train2.drop('is_ultra', axis=1)
target_train_2 = df_train2['is_ultra']
features_valid_2 = df_valid2.drop('is_ultra', axis=1)
target_valid_2 = df_valid2['is_ultra']
features_test_2 = df_test2.drop('is_ultra', axis=1)
target_test_2 = df_test2['is_ultra']

__Вывод__

Мы разбили наш датафрейм на 3 части = 60:20:20 процентов, то есть 1928 : 643 :  643 объектов соответственно тренировочная выборка, валидационная и тестовая.

Подготовили выборки к работе - отделили признаки и ответы.

## 3. Исследуйте модели

__1. RandomForestClassifier__

Исходя из опыта - одна из самых точных моделей - случайный лес. Начнем с нее свое исследование.

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

accuracy_max = 0
for estim in range(2, 50, 5):
    for depth in range(3, 20, 3):
        model_random_forest_2 = RandomForestClassifier(random_state=222, n_estimators=estim, max_depth=depth, 
                                                      criterion="entropy")
        model_random_forest_2.fit(features_train_2, target_train_2)
        predictions = model_random_forest_2.predict(features_valid_2)
        accuracy = accuracy_score(target_valid_2, predictions)
        if accuracy_max < accuracy:
            depth_max = depth
            estim_max = estim
            accuracy_max = accuracy
            
print()
print('деревьев в ансамбле:', estim_max )
print('max_depth:', depth_max )
print('accuracy:', accuracy_max )



деревьев в ансамбле: 22
max_depth: 12
accuracy: 0.8289269051321928


Опытным путём выяснили, что оптимальная количество деревьев - 22 c глубиной 12.

Попробовали разные критерии - джини и "entropy", и не увидели значимой разницы.

In [15]:
model_optim = RandomForestClassifier(random_state=222, n_estimators=estim_max, max_depth=depth_max)
model_optim.fit(features_train_2, target_train_2)

train_prediction = model_optim.predict(features_train_2)
valid_prediction = model_optim.predict(features_valid_2)

print()
print('деревьев в ансамбле:', estim_max)
print('глубина дерева:', depth_max)
print('accuracy_train:', accuracy_score(target_train_2, train_prediction))
print('accuracy_valid:', accuracy_score(target_valid_2, valid_prediction))


деревьев в ансамбле: 22
глубина дерева: 12
accuracy_train: 0.9144190871369294
accuracy_valid: 0.8164852255054432


__Посчитаем Precision и Recall.__

Для этого воспользуемся функцией confusion_matrix, которая посчитает истинно отрицательные (tn), ложно положительные (fp), ложно отрицательные (fn) и истинно положительные результаты (tp). И расчитаем искомые показатели.

In [16]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(target_valid_2, valid_prediction).ravel()
tn, fp, fn, tp

(409, 34, 84, 116)

In [17]:
precision = tp/(tp + fp)
precision

0.7733333333333333

In [18]:
recall = tp/(tp + fn)
recall

0.58

__Вывод__

Лучший результат, который я получил из модели случайного леса, экспериментируя с параметрами количества деревьев в ансамьле, глубины деревьев и критерием (джини и "entropy"):


деревьев в ансамбле: 22, глубина дерева: 12

__accuracy_tran__: 0.914

__accuracy_valid__: 0.816

__precision__: 0.773 (примерно 0,8)

__recall__: 0.58 (примерно 0,6)

__2. DecisionTreeClassifier__

Посмотрим, удастся ли добиться похожего результата, используя только одно дерево.

In [19]:
from sklearn.tree import DecisionTreeClassifier

accuracy_max = 0
for depth in range(1, 21, 1):
    model_tree = DecisionTreeClassifier(random_state = 22, max_depth=depth)
    model_tree.fit(features_train_2, target_train_2)
    predictions = model_tree.predict(features_valid_2)
    accuracy = accuracy_score(target_valid_2, predictions)
    if accuracy_max < accuracy:
        depth_max = depth
        estim_max = estim
        accuracy_max = accuracy
    
    
print('Лучший результат у дерева с параметром')
print('max_depth:', depth_max)
print('accuracy:', accuracy_max)

Лучший результат у дерева с параметром
max_depth: 8
accuracy: 0.8149300155520995


Лучший результат при глубине = 8.

Посмотрим, что изменится, если зададим параметр max_leaf_nodes

In [20]:
from sklearn.tree import DecisionTreeClassifier

accuracy_max = 0
for nodes in range(2, 21, 1):
    for depth in range(1, 20, 1):
        model_tree = DecisionTreeClassifier(random_state = 22, max_depth=depth, max_leaf_nodes=nodes)
        model_tree.fit(features_train_2, target_train_2)
        predictions = model_tree.predict(features_valid_2)
        accuracy = accuracy_score(target_valid_2, predictions)
        if accuracy_max < accuracy:
            depth_max = depth
            leaf_nodes_max = nodes
            accuracy_max = accuracy
    
    
print('Лучший результат у дерева с параметрами')
print('max_leaf_nodes:', leaf_nodes_max)
print('max_depth:', depth_max)
print('accuracy:', accuracy_max)   

Лучший результат у дерева с параметрами
max_leaf_nodes: 9
max_depth: 6
accuracy: 0.8133748055987559


Мы видим, что при незаданном параметре max_leaf_nodes, оптимальное количество деревьев - 8, но при заданном параметре max_leaf_nodes = 9, достаточно глубины = 6. При этом точность примерно равна: 0,8133 (с глубиной дерева = 6 и max_leaf_nodes = 9) и 0,8149 (с шлубиной дереа = 8, но без max_leaf_nodes). 

В дальнейшем будем использовать параметр max_leaf_nodes = 9, чтобы сократить глубину дерева.

__Посчитаем Precision и Recall.__

In [21]:
valid_prediction = model_tree.predict(features_valid_2)

In [22]:
tn, fp, fn, tp = confusion_matrix(target_valid_2, valid_prediction).ravel()
tn, fp, fn, tp

(408, 35, 91, 109)

In [23]:
precision = tp/(tp + fp)
precision

0.7569444444444444

In [24]:
recall = tp/(tp + fn)
recall

0.545

__Вывод__

Характеристики решающего дерева уступют характеристикам случайного леса.

__accuracy_valid__: 8133

__recision__: 0.7569 

__recall__: 0.545 

__3. LinearRegression__

Посмотрим на третью модель классификации.

In [25]:
from sklearn.linear_model import LogisticRegression


model_log_regr_l2 = LogisticRegression(random_state=22, penalty='l2', solver='newton-cg')
model_log_regr_l2.fit(features_train_2, target_train_2)
prediction_valid = model_log_regr_l2.predict(features_valid_2)
accuracy = accuracy_score(target_valid_2, prediction_valid)
print('accuracy:', accuracy)

prediction_valid = model_log_regr_l2.predict(features_valid_2)
tn, fp, fn, tp = confusion_matrix(target_valid_2, prediction_valid).ravel()
tn, fp, fn, tp

precision = tp/(tp + fp)
print()
print ('precision:', precision)

recall = tp/(tp + fn)
print()
print ('recall:', recall)

accuracy: 0.7480559875583204

precision: 0.8275862068965517

recall: 0.24


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line 

Таким образом увидев результаты трех моделей.


Лиейная регрессия лучше всех посчитала полноту, но по остальным показателям заметно уступила.

И так еще раз приведем результаты для наглядности и общего вывода.

__Случайный лес__

    accuracy_valid: 0.816

    precision: 0.773 (примерно 0,8)

    recall: 0.58 (примерно 0,6)
    
__Решающее дерево__

    accuracy_valid: 8133

    recision: 0.7569

    recall: 0.545
    
__Линейная регрессия__

    accuracy_valid: 0.7480559875583204

    precision: 0.8275862068965517

    recall: 0.24

Видим, что лучший результат у модели случайного леса, но решающее дерево стоит рядом, лиишь незначительно уступая. Если мы чувствительны к поглощаемым ресурсам моделью, то стоит выбрать решающее дерево для решения задач. Если у нас в приоритете стоит полнота, то отдадим предпочтение линейной регрессии. 

## 4. Проверьте модель на тестовой выборке

In [26]:
print('Модель случайного леса: деревьев в ансамбле: 22, глубина дерева: 12')
print('accuracy_test:', accuracy_score(target_test_2, model_optim.predict(features_test_2)))


Модель случайного леса: деревьев в ансамбле: 22, глубина дерева: 12
accuracy_test: 0.7962674961119751


In [27]:
predictions = model_tree.predict(features_test_2)
accuracy = accuracy_score(target_test_2, predictions)
print('Модель - решающее дерево')
print('accuracy_test:', accuracy)

Модель - решающее дерево
accuracy_test: 0.7791601866251944


In [29]:
predictions = model_log_regr_l2.predict(features_test_2)
accuracy = accuracy_score(target_test_2, predictions)
print('Модель - линейная регрессия')
print('accuracy_test:', accuracy)

Модель - линейная регрессия
accuracy_test: 0.7589424572317263


Вполне ожидаемые результаты.

Случайный лес, точность - 0,79

Решающее дерево, точность - 0,78

Линейная регрессия, точность - 0,76


## 5. Проверим модели на адекватность

Т.к. у нас данные по тарифам смещены, тариф Смарт = 70% и Ультра = 30%. Поэтому, если мы заполним массив в качестве предсказания нулями, его точность составит 70%. 

In [30]:
len(target_valid_2)

643

In [31]:
only_null = [0]*643

In [32]:
accuracy = accuracy_score(target_valid_2, only_null)
print('accuracy:', accuracy)

accuracy: 0.6889580093312597


Посмотрим сколько составит точность, если массив заполним на 70% нулями и 30% - единиц и перемешаем их?

In [33]:
mix_one_null = [0]*451 + [1]*192

In [34]:
import random

sorted(mix_one_null, key=lambda *args: random.random())

[1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,


In [35]:
accuracy = accuracy_score(target_valid_2, mix_one_null)
print('accuracy:', accuracy)

accuracy: 0.5707620528771384


Теперь мы можем сказать, что наши модели работают вполне адекватно. Измеряемая нами основная метрика качества - accuracy - это не случайный результат.